In [144]:
# Supressing the warnings.

import warnings
warnings.filterwarnings('ignore')

In [145]:
# Importing the required libraries and packages.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, cross_val_predict,RandomizedSearchCV

from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.neighbors import KNeighborsClassifier

In [146]:
df=pd.read_csv('train.csv')
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [147]:
df.drop('recruitment_channel',axis=1,inplace=True)
df.head()

,employee_id,department,region,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,1,45,3.0,2,0,0,73,0


In [148]:
df.describe(percentiles=[0.0,0.25,0.50,0.75,0.90])

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
count,54808.000000,54808.000000,54808.000000,50684.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000
mean,39195.830627,1.253011,34.803915,3.329256,5.865512,0.351974,0.023172,63.386750,0.085170
std,22586.581449,0.609264,7.660169,1.259993,4.265094,0.477590,0.150450,13.371559,0.279137
min,1.000000,1.000000,20.000000,1.000000,1.000000,0.000000,0.000000,39.000000,0.000000
0%,1.000000,1.000000,20.000000,1.000000,1.000000,0.000000,0.000000,39.000000,0.000000
25%,19669.750000,1.000000,29.000000,3.000000,3.000000,0.000000,0.000000,51.000000,0.000000
50%,39225.500000,1.000000,33.000000,3.000000,5.000000,0.000000,0.000000,60.000000,0.000000
75%,58730.500000,1.000000,39.000000,4.000000,7.000000,1.000000,0.000000,76.000000,0.000000
90%,70459.300000,2.000000,46.000000,5.000000,11.000000,1.000000,0.000000,83.000000,0.000000
max,78298.000000,10.000000,60.000000,5.000000,37.000000,1.000000,1.000000,99.000000,1.000000


In [149]:
bins = np.arange(10, df['age'].max() + 10, 10)
df['Age_bins'] = pd.cut(df['age'], bins, include_lowest=True)
df.head()

,employee_id,department,region,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins
0,65438,Sales & Marketing,region_7,Master's & above,f,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]"
1,65141,Operations,region_22,Bachelor's,m,1,30,5.0,4,0,0,60,0,"(20.0, 30.0]"
2,7513,Sales & Marketing,region_19,Bachelor's,m,1,34,3.0,7,0,0,50,0,"(30.0, 40.0]"
3,2542,Sales & Marketing,region_23,Bachelor's,m,2,39,1.0,10,0,0,50,0,"(30.0, 40.0]"
4,48945,Technology,region_26,Bachelor's,m,1,45,3.0,2,0,0,73,0,"(40.0, 50.0]"


In [150]:
df.groupby('department').sum()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
department,,,,,,,,,
Analytics,209301691,7160,173572,17381.0,26400,2163,130,452795,512
Finance,101257247,3307,81301,7900.0,12222,1052,70,152723,206
HR,96807617,2548,83145,7565.0,13743,808,45,120944,136
Legal,39642336,1132,35152,2966.0,4779,347,25,62203,53
Operations,443990620,13173,409364,39082.0,72809,4888,262,683452,1023
Procurement,282026620,9442,256254,22132.0,45191,2695,171,500534,688
R&D,38203769,1401,32810,3250.0,4550,416,21,84512,69
Sales & Marketing,658546926,21512,587053,47642.0,99563,4464,361,846407,1213
Technology,278468259,9000,248882,20822.0,42220,2458,185,570531,768


In [151]:
df.groupby('department').mean()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
department,,,,,,,,,
Analytics,39107.191891,1.337818,32.431241,3.557307,4.932735,0.404148,0.024290,84.602952,0.095665
Finance,39927.936514,1.304022,32.058754,3.506436,4.819401,0.414826,0.027603,60.222003,0.081230
HR,40036.235318,1.053763,34.385856,3.482965,5.683623,0.334160,0.018610,50.018197,0.056245
Legal,38154.317613,1.089509,33.832531,3.405281,4.599615,0.333975,0.024062,59.868142,0.051011
Operations,39125.010575,1.160821,36.073669,3.632156,6.416020,0.430737,0.023088,60.226648,0.090148
Procurement,39510.594004,1.322779,35.899972,3.298852,6.331045,0.377557,0.023956,70.122443,0.096386
R&D,38242.011011,1.402402,32.842843,3.563596,4.554555,0.416416,0.021021,84.596597,0.069069
Sales & Marketing,39106.111995,1.277435,34.860629,3.067937,5.912292,0.265083,0.021437,50.261698,0.072031
Technology,39012.084477,1.260857,34.867190,3.158677,5.914822,0.344354,0.025918,79.928692,0.107593


In [152]:
df.groupby('is_promoted').mean()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
is_promoted,,,,,,,,
0,39187.520423,1.257639,34.844037,3.267911,5.879398,0.319685,0.014180,62.647686
1,39285.092331,1.203299,34.372965,3.986140,5.716367,0.698800,0.119751,71.325193


In [153]:
df_prmt=df[df['is_promoted']==1]

In [154]:
df_prmt.head()

,employee_id,department,region,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins
11,49017,Sales & Marketing,region_7,Bachelor's,f,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]"
39,58304,Sales & Marketing,region_28,Bachelor's,m,1,33,5.0,6,1,0,51,1,"(30.0, 40.0]"
60,17673,Sales & Marketing,region_4,Master's & above,m,1,50,4.0,17,1,0,47,1,"(40.0, 50.0]"
66,77981,Finance,region_22,Bachelor's,m,1,27,NaN,1,1,1,58,1,"(20.0, 30.0]"
67,16502,Sales & Marketing,region_22,Bachelor's,m,1,27,NaN,1,0,0,61,1,"(20.0, 30.0]"


In [155]:
df_prmt.groupby('department').mean()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
department,,,,,,,,,
Analytics,39428.859375,1.384766,31.623047,4.025641,4.746094,0.693359,0.119141,85.833984,1.0
Finance,41767.067961,1.213592,30.747573,3.950549,4.135922,0.631068,0.140777,68.631068,1.0
HR,38886.588235,1.058824,34.000000,4.106557,5.536765,0.698529,0.132353,57.897059,1.0
Legal,39172.603774,1.132075,31.905660,4.023810,4.113208,0.735849,0.132075,66.415094,1.0
Operations,39477.823069,1.219941,36.406647,4.032454,6.601173,0.682307,0.122190,67.221896,1.0
Procurement,38864.359012,1.140988,35.640988,4.042945,6.297965,0.726744,0.119186,75.989826,1.0
R&D,36918.420290,1.391304,31.971014,4.203390,3.797101,0.724638,0.086957,85.666667,1.0
Sales & Marketing,39073.606760,1.133553,34.489695,3.877240,5.670239,0.708986,0.117065,59.932399,1.0
Technology,39268.680990,1.236979,33.601562,3.982906,5.475260,0.696615,0.115885,83.084635,1.0


In [156]:
df_prmt['department'].value_counts()

Sales & Marketing    1213
Operations           1023
Technology            768
Procurement           688
Analytics             512
Finance               206
HR                    136
R&D                    69
Legal                  53
Name: department, dtype: int64

In [157]:
department_mean=pd.DataFrame(round(df_prmt['department'].value_counts()/df['department'].value_counts(),4))
department_mean

,department
Analytics,0.0957
Finance,0.0812
HR,0.0562
Legal,0.0510
Operations,0.0901
Procurement,0.0964
R&D,0.0691
Sales & Marketing,0.0720
Technology,0.1076


In [158]:
dpt=department_mean.reset_index()

In [159]:
dpt.columns

Index(['index', 'department'], dtype='object')

In [160]:
dpt['department'] = dpt['department'].astype(np.object)
data = df.merge(dpt, left_on = 'department', right_on = 'index')
data.head()

,employee_id,department_x,region,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,index,department_y
0,65438,Sales & Marketing,region_7,Master's & above,f,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",Sales & Marketing,0.072
1,7513,Sales & Marketing,region_19,Bachelor's,m,1,34,3.0,7,0,0,50,0,"(30.0, 40.0]",Sales & Marketing,0.072
2,2542,Sales & Marketing,region_23,Bachelor's,m,2,39,1.0,10,0,0,50,0,"(30.0, 40.0]",Sales & Marketing,0.072
3,28911,Sales & Marketing,region_1,Master's & above,m,1,32,5.0,5,1,0,54,0,"(30.0, 40.0]",Sales & Marketing,0.072
4,49017,Sales & Marketing,region_7,Bachelor's,f,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",Sales & Marketing,0.072


In [161]:
rgn_mean=pd.DataFrame(round(df_prmt['region'].value_counts()/df['region'].value_counts(),4))
rgn_mean.head()

,region
region_1,0.0951
region_10,0.0787
region_11,0.0563
region_12,0.0660
region_13,0.0869


In [162]:
rgn_mean.reset_index(inplace=True)
rgn_mean.head()

,index,region
0,region_1,0.0951
1,region_10,0.0787
2,region_11,0.0563
3,region_12,0.0660
4,region_13,0.0869


In [163]:
rgn_mean

,index,region
0,region_1,0.0951
1,region_10,0.0787
2,region_11,0.0563
3,region_12,0.0660
4,region_13,0.0869
5,region_14,0.0750
6,region_15,0.0791
7,region_16,0.0696
8,region_17,0.1369
9,region_18,0.0323


In [164]:
rgn_mean['region'] = rgn_mean['region'].astype(np.object)
data = data.merge(rgn_mean, left_on = 'region', right_on = 'index')
data.head()

,employee_id,department_x,region_x,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,index_x,department_y,index_y,region_y
0,65438,Sales & Marketing,region_7,Master's & above,f,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",Sales & Marketing,0.072,region_7,0.1065
1,49017,Sales & Marketing,region_7,Bachelor's,f,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",Sales & Marketing,0.072,region_7,0.1065
2,35465,Sales & Marketing,region_7,NaN,f,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",Sales & Marketing,0.072,region_7,0.1065
3,47123,Sales & Marketing,region_7,Bachelor's,f,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",Sales & Marketing,0.072,region_7,0.1065
4,77577,Sales & Marketing,region_7,Bachelor's,m,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",Sales & Marketing,0.072,region_7,0.1065


In [165]:
data.drop(['department_x','region_x','index_x','index_y'],axis=1,inplace=True)
data.head()

,employee_id,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,department_y,region_y
0,65438,Master's & above,f,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065
1,49017,Bachelor's,f,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065
2,35465,NaN,f,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",0.072,0.1065
3,47123,Bachelor's,f,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065
4,77577,Bachelor's,m,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",0.072,0.1065


In [166]:
data['education']=data['education'].fillna(value="Bachelor's")
data.head()

,employee_id,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,department_y,region_y
0,65438,Master's & above,f,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065
1,49017,Bachelor's,f,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065
2,35465,Bachelor's,f,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",0.072,0.1065
3,47123,Bachelor's,f,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065
4,77577,Bachelor's,m,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",0.072,0.1065


In [167]:
data.isnull().sum()

employee_id                0
education                  0
gender                     0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
Age_bins                   0
department_y               0
region_y                   0
dtype: int64

In [168]:
data['previous_year_rating']=data['previous_year_rating'].fillna(value=0)
data.head()

,employee_id,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,department_y,region_y
0,65438,Master's & above,f,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065
1,49017,Bachelor's,f,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065
2,35465,Bachelor's,f,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",0.072,0.1065
3,47123,Bachelor's,f,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065
4,77577,Bachelor's,m,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",0.072,0.1065


In [169]:
data['total_points']=data['no_of_trainings']*data['avg_training_score']
data.head()

,employee_id,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,department_y,region_y,total_points
0,65438,Master's & above,f,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065,49
1,49017,Bachelor's,f,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065,50
2,35465,Bachelor's,f,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",0.072,0.1065,48
3,47123,Bachelor's,f,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065,47
4,77577,Bachelor's,m,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",0.072,0.1065,52


In [170]:
data['education'].value_counts()

Bachelor's          39078
Master's & above    14925
Below Secondary       805
Name: education, dtype: int64

In [171]:
data['education_lbl']=data['education'].map({"Bachelor's":2,"Master's & above":1,'Below Secondary':3})
data.head()

,employee_id,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,department_y,region_y,total_points,education_lbl
0,65438,Master's & above,f,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065,49,1
1,49017,Bachelor's,f,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065,50,2
2,35465,Bachelor's,f,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",0.072,0.1065,48,2
3,47123,Bachelor's,f,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065,47,2
4,77577,Bachelor's,m,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",0.072,0.1065,52,2


In [172]:
data['gender_lbl']=data['gender'].map({'f':1,'m':0})
data.head()

,employee_id,education,gender,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,department_y,region_y,total_points,education_lbl,gender_lbl
0,65438,Master's & above,f,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065,49,1,1
1,49017,Bachelor's,f,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065,50,2,1
2,35465,Bachelor's,f,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",0.072,0.1065,48,2,1
3,47123,Bachelor's,f,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065,47,2,1
4,77577,Bachelor's,m,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",0.072,0.1065,52,2,0


In [173]:
data.drop(['education','gender'],axis=1,inplace=True)
data.head()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,department_y,region_y,total_points,education_lbl,gender_lbl
0,65438,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065,49,1,1
1,49017,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065,50,2,1
2,35465,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",0.072,0.1065,48,2,1
3,47123,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065,47,2,1
4,77577,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",0.072,0.1065,52,2,0


In [174]:
data['KPI_OTH']=data['previous_year_rating']+data['KPIs_met >80%']+data['awards_won?']
data.head()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,department_y,region_y,total_points,education_lbl,gender_lbl,KPI_OTH
0,65438,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065,49,1,1,6.0
1,49017,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065,50,2,1,6.0
2,35465,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",0.072,0.1065,48,2,1,1.0
3,47123,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065,47,2,1,5.0
4,77577,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",0.072,0.1065,52,2,0,3.0


In [176]:
data['joining_age']=data['age']-data['length_of_service']
data.head()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins,department_y,region_y,total_points,education_lbl,gender_lbl,KPI_OTH,joining_age
0,65438,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065,49,1,1,6.0,27
1,49017,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065,50,2,1,6.0,32
2,35465,1,24,1.0,2,0,0,48,0,"(20.0, 30.0]",0.072,0.1065,48,2,1,1.0,22
3,47123,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065,47,2,1,5.0,37
4,77577,1,50,3.0,14,0,0,52,0,"(40.0, 50.0]",0.072,0.1065,52,2,0,3.0,36


In [180]:
age_mean=pd.DataFrame(round(df_prmt['Age_bins'].value_counts()/df['Age_bins'].value_counts(),4))
age_mean.reset_index(inplace=True)
age_mean

,index,Age_bins
0,"(30.0, 40.0]",0.0906
1,"(20.0, 30.0]",0.0859
2,"(40.0, 50.0]",0.0727
3,"(50.0, 60.0]",0.0670
4,"(9.999, 20.0]",0.0354


In [181]:
age_mean['Age_bins'] = age_mean['Age_bins'] .astype(np.object)
data = data.merge(age_mean, left_on = 'Age_bins', right_on = 'index')
data.head()

,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,Age_bins_x,department_y,region_y,total_points,education_lbl,gender_lbl,KPI_OTH,joining_age,index,Age_bins_y
0,65438,1,35,5.0,8,1,0,49,0,"(30.0, 40.0]",0.072,0.1065,49,1,1,6.0,27,"(30.0, 40.0]",0.0906
1,49017,1,35,5.0,3,1,0,50,1,"(30.0, 40.0]",0.072,0.1065,50,2,1,6.0,32,"(30.0, 40.0]",0.0906
2,47123,1,40,5.0,3,0,0,47,0,"(30.0, 40.0]",0.072,0.1065,47,2,1,5.0,37,"(30.0, 40.0]",0.0906
3,28509,1,40,5.0,8,1,0,54,0,"(30.0, 40.0]",0.072,0.1065,54,2,0,6.0,32,"(30.0, 40.0]",0.0906
4,14141,1,40,3.0,2,1,0,90,1,"(30.0, 40.0]",0.072,0.1065,90,2,1,4.0,38,"(30.0, 40.0]",0.0906


In [182]:
data.drop(['age','Age_bins_x','index'],axis=1,inplace=True)
data.head()

,employee_id,no_of_trainings,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,department_y,region_y,total_points,education_lbl,gender_lbl,KPI_OTH,joining_age,Age_bins_y
0,65438,1,5.0,8,1,0,49,0,0.072,0.1065,49,1,1,6.0,27,0.0906
1,49017,1,5.0,3,1,0,50,1,0.072,0.1065,50,2,1,6.0,32,0.0906
2,47123,1,5.0,3,0,0,47,0,0.072,0.1065,47,2,1,5.0,37,0.0906
3,28509,1,5.0,8,1,0,54,0,0.072,0.1065,54,2,0,6.0,32,0.0906
4,14141,1,3.0,2,1,0,90,1,0.072,0.1065,90,2,1,4.0,38,0.0906


In [187]:
    data.rename(columns={'KPIs_met >80%':'KPIs','awards_won?':'awards_won','department_y':'department_mean',
                         'region_y':'region_mean','Age_bins_y':'Age_bins'},inplace=True)
    data.head()

,employee_id,no_of_trainings,previous_year_rating,length_of_service,KPIs,awards_won,avg_training_score,is_promoted,department_mean,region_mean,total_points,education_lbl,gender_lbl,KPI_OTH,joining_age,Age_bins
0,65438,1,5.0,8,1,0,49,0,0.072,0.1065,49,1,1,6.0,27,0.0906
1,49017,1,5.0,3,1,0,50,1,0.072,0.1065,50,2,1,6.0,32,0.0906
2,47123,1,5.0,3,0,0,47,0,0.072,0.1065,47,2,1,5.0,37,0.0906
3,28509,1,5.0,8,1,0,54,0,0.072,0.1065,54,2,0,6.0,32,0.0906
4,14141,1,3.0,2,1,0,90,1,0.072,0.1065,90,2,1,4.0,38,0.0906


In [188]:
X= data.drop(['employee_id','is_promoted'],axis=1)
y=df['is_promoted']

In [190]:
# Splitting the dataset into train-test.

from sklearn import model_selection

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, stratify = y, shuffle = True, random_state = 100)

In [191]:
# Looking at the dimensions of the train and test sets.

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(43846, 14)
(10962, 14)
(43846,)
(10962,)


In [192]:
# looking at the proportion of target variable in train and test sets.

print(np.sum(y))
print(np.sum(y_train))
print(np.sum(y_test))

4668
3734
934


In [193]:
from sklearn.preprocessing import StandardScaler 
sc=StandardScaler()
X_train_std=sc.fit_transform(X_train)

In [194]:
X_train_std[:10]

array([[ 2.85233505, -0.05083888,  0.03321574,  1.35292929, -0.15339658,
         1.16867652,  1.52777467, -1.65267794,  3.56577214,  0.54494515,
        -0.65110774,  0.32154986, -0.50287533,  0.71148211],
       [-0.41601678, -0.05083888,  0.03321574,  1.35292929, -0.15339658,
         1.39317046,  1.52777467, -0.1935433 ,  0.04829758,  0.54494515,
         1.53584414,  0.32154986, -0.67368052,  0.71148211],
       [-0.41601678, -0.05083888, -0.90653421, -0.73913693, -0.15339658,
         1.0938452 ,  0.71762766,  0.82553487, -0.04247595,  0.54494515,
        -0.65110774, -0.26557691, -1.01529091,  0.10005916],
       [-0.41601678,  1.28634273, -0.90653421, -0.73913693, -0.15339658,
        -0.4027811 , -0.26952626,  0.82553487, -0.49634364,  0.54494515,
        -0.65110774,  0.90867664, -0.50287533,  0.10005916],
       [-0.41601678, -1.3880205 ,  0.26815323,  1.35292929, -0.15339658,
         1.61766441,  0.71762766, -0.1935433 ,  0.11637774,  0.54494515,
        -0.65110774, -0.85

In [195]:
X_train_std=pd.DataFrame(X_train_std)
col=X_train.columns
X_train_std.columns=col
X_train_std.head()

,no_of_trainings,previous_year_rating,length_of_service,KPIs,awards_won,avg_training_score,department_mean,region_mean,total_points,education_lbl,gender_lbl,KPI_OTH,joining_age,Age_bins
0,2.852335,-0.050839,0.033216,1.352929,-0.153397,1.168677,1.527775,-1.652678,3.565772,0.544945,-0.651108,0.321550,-0.502875,0.711482
1,-0.416017,-0.050839,0.033216,1.352929,-0.153397,1.393170,1.527775,-0.193543,0.048298,0.544945,1.535844,0.321550,-0.673681,0.711482
2,-0.416017,-0.050839,-0.906534,-0.739137,-0.153397,1.093845,0.717628,0.825535,-0.042476,0.544945,-0.651108,-0.265577,-1.015291,0.100059
3,-0.416017,1.286343,-0.906534,-0.739137,-0.153397,-0.402781,-0.269526,0.825535,-0.496344,0.544945,-0.651108,0.908677,-0.502875,0.100059
4,-0.416017,-1.388020,0.268153,1.352929,-0.153397,1.617664,0.717628,-0.193543,0.116378,0.544945,-0.651108,-0.852704,-1.186096,0.100059


In [196]:
X_test_std=sc.transform(X_test)
X_test_std[:10]

array([[ 1.21815914, -0.05083888, -0.67159672, -0.73913693, -0.15339658,
        -1.30075688, -0.89585841, -0.59885848,  0.27523143,  0.54494515,
        -0.65110774, -0.26557691,  0.35115064,  0.71148211],
       [-0.41601678,  1.28634273, -0.20172174,  1.35292929, -0.15339658,
        -0.85176899, -0.89585841, -0.16266214, -0.63250394, -1.58177182,
        -0.65110774,  1.49580342, -0.50287533,  0.71148211],
       [-0.41601678,  0.61775192,  0.73802821,  1.35292929, -0.15339658,
         1.54283309,  0.71762766, -0.1935433 ,  0.09368435,  0.54494515,
         1.53584414,  0.90867664,  0.52195583, -1.61712872],
       [-0.41601678, -0.05083888, -0.67159672,  1.35292929, -0.15339658,
         1.46800178,  1.52777467, -0.1935433 ,  0.07099097,  0.54494515,
         1.53584414,  0.32154986,  4.45047529, -2.35864166],
       [-0.41601678, -0.05083888, -0.43665923,  1.35292929, -0.15339658,
        -0.55244373, -0.89585841, -0.59885848, -0.54173041,  0.54494515,
        -0.65110774,  0.32

In [197]:
X_test_std=pd.DataFrame(X_test_std)
col=X_test.columns
X_test_std.columns=col
X_test_std.head()

,no_of_trainings,previous_year_rating,length_of_service,KPIs,awards_won,avg_training_score,department_mean,region_mean,total_points,education_lbl,gender_lbl,KPI_OTH,joining_age,Age_bins
0,1.218159,-0.050839,-0.671597,-0.739137,-0.153397,-1.300757,-0.895858,-0.598858,0.275231,0.544945,-0.651108,-0.265577,0.351151,0.711482
1,-0.416017,1.286343,-0.201722,1.352929,-0.153397,-0.851769,-0.895858,-0.162662,-0.632504,-1.581772,-0.651108,1.495803,-0.502875,0.711482
2,-0.416017,0.617752,0.738028,1.352929,-0.153397,1.542833,0.717628,-0.193543,0.093684,0.544945,1.535844,0.908677,0.521956,-1.617129
3,-0.416017,-0.050839,-0.671597,1.352929,-0.153397,1.468002,1.527775,-0.193543,0.070991,0.544945,1.535844,0.321550,4.450475,-2.358642
4,-0.416017,-0.050839,-0.436659,1.352929,-0.153397,-0.552444,-0.895858,-0.598858,-0.541730,0.544945,-0.651108,0.321550,-0.502875,0.100059


In [198]:
# Hyperparameter tuning for Logistic Regression model.

log_reg_params = {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid = log_reg_params, cv = 8, n_jobs = -1, verbose = 1,scoring='f1')

In [199]:
# Fitting the Grid Search to the training dataset.

grid_search.fit(X_train_std, y_train)

Fitting 8 folds for each of 14 candidates, totalling 112 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed:    5.9s finished


GridSearchCV(cv=8, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='f1', verbose=1)

In [200]:
grid_search.best_params_

{'C': 0.001, 'penalty': 'l2'}

In [201]:
# Building a model with the best hyperparameters.

log_reg_imb_1 = LogisticRegression(C = 0.001, penalty = 'l2')

# Fitting on the training dataset.

log_reg_imb_1.fit(X_train_std, y_train)

LogisticRegression(C=0.001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [202]:
# Making predictions on the test dataset.

y_pred_log_imb= log_reg_imb_1.predict(X_test_std)
y_pred_log_imb[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [203]:
metrics.confusion_matrix(y_test, y_pred_log_imb)

array([[10028,     0],
       [  934,     0]], dtype=int64)

In [204]:
# Creating the parameter grid.

param_grid = {'max_depth': [3, 4, 5], 'min_samples_leaf': range(100, 400, 200), 'min_samples_split': range(100, 400, 200),
              'n_estimators': [100, 200, 300], 'max_features': [10, 12, 14]}

# Instantiating the random forest model.

rf_tune = RandomForestClassifier(random_state=100)

# Instantiating the Grid Search model.

grid_search = GridSearchCV(estimator = rf_tune, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 1,scoring='f1')

In [205]:
# Fitting the Grid Search to the training dataset.

grid_search.fit(X_train_std, y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 324 out of 324 | elapsed:  9.4min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=100,
                                 

In [206]:
grid_search.best_params_

{'max_depth': 3,
 'max_features': 10,
 'min_samples_leaf': 100,
 'min_samples_split': 100,
 'n_estimators': 100}

In [207]:
rf_tune_imb = RandomForestClassifier(bootstrap=True, max_depth=3, max_features=10, min_samples_leaf=100, 
                                  min_samples_split=100, n_estimators=100, random_state=100)
rf_tune_imb.fit(X_train_std,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features=10,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=100, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=100,
                       verbose=0, warm_start=False)

In [208]:
y_pred_rf=rf_tune_imb.predict(X_test_std)
y_pred_rf[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [209]:
metrics.confusion_matrix(y_test, y_pred_rf)

array([[10028,     0],
       [  934,     0]], dtype=int64)

In [210]:
# Specifying the range of hyperparameters.

param_grid = {'learning_rate': [0.2, 0.5], 'subsample': [0.3, 0.6, 0.9], 'max_depth': [3, 4, 5, 6], 'n_estimators': [100, 200, 300]}          

# Specifying the model.

xgb_tune = XGBClassifier(random_state=100)

# Setting up GridSearchCV().

grid_search = GridSearchCV(estimator = xgb_tune, param_grid = param_grid, scoring = 'f1', cv = 3, verbose = 1,
                        return_train_score = True, n_jobs = -1) 

In [212]:
# Fitting the Grid Search to the training dataset.

grid_search.fit(X_train_std,y_train)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   49.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  7.0min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=100, reg_alpha=0,
                                     reg_lambda=1, scale_pos_weight=1,
                                     seed=None, silent=None, subsample=1,
                                     verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.2, 0.5], 'max_depth': [3, 4, 5, 6],
                         'n_estimators': [100, 200, 300],
 

In [213]:
grid_search.best_params_

{'learning_rate': 0.5, 'max_depth': 6, 'n_estimators': 300, 'subsample': 0.3}

In [214]:
# Building a model with the best hyperparameters.

xgb_final_1_imb = XGBClassifier(learning_rate = 0.5, max_depth = 6, n_estimators = 300, subsample = 0.3)

# Fitting on the training dataset.

xgb_final_1_imb.fit(X_train_std,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.5, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.3, verbosity=1)

In [216]:
y_pred_xgb_imb=xgb_final_1_imb.predict(X_test_std)
y_pred_xgb_imb[:10]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int64)

In [217]:
# Looking at the confusion matrix.

metrics.confusion_matrix(y_test, y_pred_xgb_imb)

array([[9634,  394],
       [ 899,   35]], dtype=int64)

In [218]:
from imblearn.over_sampling import SMOTE
X_train_smote, y_train_smote = SMOTE(random_state = 27).fit_resample(X_train_std, y_train)

In [219]:
# Specifying the range of hyperparameters.

param_grid = {'learning_rate': [0.2, 0.5], 'subsample': [0.3, 0.6, 0.9], 'max_depth': [3, 4, 5, 6], 'n_estimators': [100, 200, 300]}          

# Specifying the model.

xgb_tune = XGBClassifier(random_state=100)

# Setting up GridSearchCV().

grid_search = GridSearchCV(estimator = xgb_tune, param_grid = param_grid, scoring = 'f1', cv = 3, verbose = 1,
                        return_train_score = True, n_jobs = -1) 

In [220]:
# Fitting the Grid Search to the training dataset.

grid_search.fit(X_train_smote,y_train_smote)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed: 14.3min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=100, reg_alpha=0,
                                     reg_lambda=1, scale_pos_weight=1,
                                     seed=None, silent=None, subsample=1,
                                     verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.2, 0.5], 'max_depth': [3, 4, 5, 6],
                         'n_estimators': [100, 200, 300],
 

In [221]:
grid_search.best_params_

{'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.9}

In [222]:
# Building a model with the best hyperparameters.

xgb_final_1_smt = XGBClassifier(learning_rate = 0.2, max_depth = 5, n_estimators = 300, subsample = 0.9)

# Fitting on the training dataset.

xgb_final_1_smt.fit(X_train_smote,y_train_smote)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1)

In [224]:
y_pred_xgb_smt=xgb_final_1_smt.predict(X_test_std)
y_pred_xgb_smt[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [225]:
# Looking at the confusion matrix.

metrics.confusion_matrix(y_test, y_pred_xgb_smt)

array([[10025,     3],
       [  934,     0]], dtype=int64)

In [227]:
from imblearn.under_sampling import RandomUnderSampler
X_train_random, y_train_random = RandomUnderSampler(random_state = 27).fit_resample(X_train_std, y_train)

In [228]:
# Specifying the range of hyperparameters.

param_grid = {'learning_rate': [0.2, 0.5], 'subsample': [0.3, 0.6, 0.9], 'max_depth': [3, 4, 5, 6], 'n_estimators': [100, 200, 300]}          

# Specifying the model.

xgb_tune = XGBClassifier(random_state=100)

# Setting up GridSearchCV().

grid_search = GridSearchCV(estimator = xgb_tune, param_grid = param_grid, scoring = 'f1', cv = 3, verbose = 1,
                        return_train_score = True, n_jobs = -1) 

In [229]:
# Fitting the Grid Search to the training dataset.

grid_search.fit(X_train_random, y_train_random)

Fitting 3 folds for each of 72 candidates, totalling 216 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-1)]: Done 216 out of 216 | elapsed:  1.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=100, reg_alpha=0,
                                     reg_lambda=1, scale_pos_weight=1,
                                     seed=None, silent=None, subsample=1,
                                     verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.2, 0.5], 'max_depth': [3, 4, 5, 6],
                         'n_estimators': [100, 200, 300],
 

In [230]:
grid_search.best_params_

{'learning_rate': 0.5, 'max_depth': 6, 'n_estimators': 300, 'subsample': 0.3}

In [231]:
# Building a model with the best hyperparameters.

xgb_final_1_rndm = XGBClassifier(learning_rate = 0.5, max_depth = 6, n_estimators = 300, subsample = 0.3)

# Fitting on the training dataset.

xgb_final_1_rndm.fit(X_train_random, y_train_random)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.5, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.3, verbosity=1)

In [232]:
y_pred_xgb_rndm=xgb_final_1_rndm.predict(X_test_std)
y_pred_xgb_rndm[:10]

array([1, 0, 1, 1, 1, 0, 0, 0, 0, 0], dtype=int64)

In [233]:
# Looking at the confusion matrix.

metrics.confusion_matrix(y_test, y_pred_xgb_rndm)

array([[5145, 4883],
       [ 470,  464]], dtype=int64)

In [234]:
# Looking at the confusion matrix.

metrics.f1_score(y_test, y_pred_xgb_rndm)

0.14774717401687631

In [235]:
import lightgbm as lgb

In [237]:
# Specifying the range of hyperparameters.

param_grid = {'learning_rate': [0.1,0.2, 0.5], 'subsample': [0.3, 0.6, 0.9], 'max_depth': [3, 4, 5, 6], 'n_estimators': [100, 200, 300],
              'min_child_samples':range(10,50,15),'min_child_weight':[0.001,0.01,0.1,1]}          

# Specifying the model.

lgbm_tune = lgb.LGBMClassifier(random_state=100)

# Setting up GridSearchCV().

grid_search = GridSearchCV(estimator = lgbm_tune, param_grid = param_grid, scoring = 'roc_auc', cv = 5, verbose = 1,
                        return_train_score = True, n_jobs = -1) 

In [238]:
grid_search.fit(X_train_std,y_train)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   34.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 15.5min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 19.0min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed: 23.4min
[Parallel(n_jobs=-1)]: Done 6480 out of 6480 | elapsed: 25.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=100, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.2, 0.5],
                         'max_depth': 

In [239]:
grid_search.best_params_

{'learning_rate': 0.5,
 'max_depth': 6,
 'min_child_samples': 10,
 'min_child_weight': 0.1,
 'n_estimators': 200,
 'subsample': 0.3}

In [240]:
lgbm_final=lgb.LGBMClassifier(random_state=100,learning_rate='0.5',max_depth='6',min_child_samples='10',min_child_weight=0.1
                             ,n_estimators=200,subsample=0.3)
lgbm_final.fit(X_train_std,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate='0.5', max_depth='6',
               min_child_samples='10', min_child_weight=0.1, min_split_gain=0.0,
               n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
               random_state=100, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.3, subsample_for_bin=200000, subsample_freq=0)

In [241]:
y_pred_lgbm_final=lgbm_final.predict(X_test_std)
y_pred_lgbm_final[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [242]:
# Looking at the confusion matrix.

metrics.confusion_matrix(y_test, y_pred_lgbm_final)

array([[9940,   88],
       [ 923,   11]], dtype=int64)

In [243]:
df_test=pd.read_csv('test.csv')
df_test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [244]:
df_test['previous_year_rating']=df_test['previous_year_rating'].fillna(value=0)
df_test['education']=df_test['education'].fillna(value="Bachelor's")
df_test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,0.0,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [245]:
data_test = df_test.merge(dpt, left_on = 'department', right_on = 'index')
data_test.head()

,employee_id,department_x,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,index,department_y
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,0.0,1,1,0,77,Technology,0.1076
1,66174,Technology,region_7,Master's & above,m,other,1,51,4.0,11,0,0,75,Technology,0.1076
2,76303,Technology,region_22,Bachelor's,m,sourcing,1,29,5.0,2,1,0,76,Technology,0.1076
3,5677,Technology,region_17,Bachelor's,m,sourcing,1,25,0.0,1,0,0,80,Technology,0.1076
4,60889,Technology,region_29,Master's & above,m,sourcing,1,30,1.0,3,0,0,83,Technology,0.1076


In [246]:
bins = np.arange(10, df['age'].max() + 10, 10)
data_test['Age_bins'] = pd.cut(data_test['age'], bins, include_lowest=True)
data_test = data_test.merge(age_mean, left_on = 'Age_bins', right_on = 'index')
data_test.head()

,employee_id,department_x,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,index_x,department_y,Age_bins_x,index_y,Age_bins_y
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,0.0,1,1,0,77,Technology,0.1076,"(20.0, 30.0]","(20.0, 30.0]",0.0859
1,76303,Technology,region_22,Bachelor's,m,sourcing,1,29,5.0,2,1,0,76,Technology,0.1076,"(20.0, 30.0]","(20.0, 30.0]",0.0859
2,5677,Technology,region_17,Bachelor's,m,sourcing,1,25,0.0,1,0,0,80,Technology,0.1076,"(20.0, 30.0]","(20.0, 30.0]",0.0859
3,60889,Technology,region_29,Master's & above,m,sourcing,1,30,1.0,3,0,0,83,Technology,0.1076,"(20.0, 30.0]","(20.0, 30.0]",0.0859
4,67672,Technology,region_17,Bachelor's,m,other,1,29,0.0,1,1,0,85,Technology,0.1076,"(20.0, 30.0]","(20.0, 30.0]",0.0859


In [247]:
data_test = data_test.merge(rgn_mean, left_on = 'region', right_on = 'index')
data_test.drop(['department_x','region_x','index_x','index_y'],axis=1,inplace=True)
data_test['education_lbl']=data_test['education'].map({"Bachelor's":2,"Master's & above":1,'Below Secondary':3})
data_test['total_points']=data_test['no_of_trainings']*data_test['avg_training_score']
data_test['gender_lbl']=data_test['gender'].map({'f':1,'m':0})
data_test['KPI_OTH']=data_test['previous_year_rating']+data_test['KPIs_met >80%']+data_test['awards_won?']
data_test['joining_age']=data_test['age']-data_test['length_of_service']
data_test.head()

,employee_id,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,department_y,Age_bins_x,Age_bins_y,index,region_y,education_lbl,total_points,gender_lbl,KPI_OTH,joining_age
0,8724,Bachelor's,m,sourcing,1,24,0.0,1,1,0,77,0.1076,"(20.0, 30.0]",0.0859,region_26,0.0633,2,77,0,1.0,23
1,14669,Bachelor's,m,referred,1,27,0.0,1,0,0,79,0.1076,"(20.0, 30.0]",0.0859,region_26,0.0633,2,79,0,0.0,26
2,21160,Bachelor's,m,sourcing,1,26,5.0,4,1,0,94,0.1076,"(20.0, 30.0]",0.0859,region_26,0.0633,2,94,0,6.0,22
3,4730,Bachelor's,f,other,1,29,4.0,2,1,0,85,0.1076,"(20.0, 30.0]",0.0859,region_26,0.0633,2,85,1,5.0,27
4,25879,Bachelor's,f,other,1,30,4.0,3,1,0,79,0.1076,"(20.0, 30.0]",0.0859,region_26,0.0633,2,79,1,5.0,27


In [248]:
data_test.drop(['age','Age_bins_x','index'],axis=1,inplace=True)
data_test.rename(columns={'KPIs_met >80%':'KPIs','awards_won?':'awards_won','department_y':'department_mean',
                     'region_y':'region_mean','Age_bins_y':'Age_bins'},inplace=True)
data_test.head()

,employee_id,education,gender,recruitment_channel,no_of_trainings,previous_year_rating,length_of_service,KPIs,awards_won,avg_training_score,department_mean,Age_bins,region_mean,education_lbl,total_points,gender_lbl,KPI_OTH,joining_age
0,8724,Bachelor's,m,sourcing,1,0.0,1,1,0,77,0.1076,0.0859,0.0633,2,77,0,1.0,23
1,14669,Bachelor's,m,referred,1,0.0,1,0,0,79,0.1076,0.0859,0.0633,2,79,0,0.0,26
2,21160,Bachelor's,m,sourcing,1,5.0,4,1,0,94,0.1076,0.0859,0.0633,2,94,0,6.0,22
3,4730,Bachelor's,f,other,1,4.0,2,1,0,85,0.1076,0.0859,0.0633,2,85,1,5.0,27
4,25879,Bachelor's,f,other,1,4.0,3,1,0,79,0.1076,0.0859,0.0633,2,79,1,5.0,27


In [249]:
data_test.drop(['education','gender','recruitment_channel'],axis=1,inplace=True)
data_test.head()

,employee_id,no_of_trainings,previous_year_rating,length_of_service,KPIs,awards_won,avg_training_score,department_mean,Age_bins,region_mean,education_lbl,total_points,gender_lbl,KPI_OTH,joining_age
0,8724,1,0.0,1,1,0,77,0.1076,0.0859,0.0633,2,77,0,1.0,23
1,14669,1,0.0,1,0,0,79,0.1076,0.0859,0.0633,2,79,0,0.0,26
2,21160,1,5.0,4,1,0,94,0.1076,0.0859,0.0633,2,94,0,6.0,22
3,4730,1,4.0,2,1,0,85,0.1076,0.0859,0.0633,2,85,1,5.0,27
4,25879,1,4.0,3,1,0,79,0.1076,0.0859,0.0633,2,79,1,5.0,27


In [251]:
X_outcome=data_test[X_train_std.columns]
X_outcome.head()

,no_of_trainings,previous_year_rating,length_of_service,KPIs,awards_won,avg_training_score,department_mean,region_mean,total_points,education_lbl,gender_lbl,KPI_OTH,joining_age,Age_bins
0,1,0.0,1,1,0,77,0.1076,0.0633,77,2,0,1.0,23,0.0859
1,1,0.0,1,0,0,79,0.1076,0.0633,79,2,0,0.0,26,0.0859
2,1,5.0,4,1,0,94,0.1076,0.0633,94,2,0,6.0,22,0.0859
3,1,4.0,2,1,0,85,0.1076,0.0633,85,2,1,5.0,27,0.0859
4,1,4.0,3,1,0,79,0.1076,0.0633,79,2,1,5.0,27,0.0859


In [253]:
X_outcome_std=sc.transform(X_outcome)
X_outcome_std[:10]

array([[-0.41601678, -2.05661131, -1.1414717 ,  1.35292929, -0.15339658,
         1.01901389,  1.52777467, -0.84204758, -0.06516934,  0.54494515,
        -0.65110774, -1.43983047, -1.01529091,  0.10005916],
       [-0.41601678, -2.05661131, -1.1414717 , -0.73913693, -0.15339658,
         1.16867652,  1.52777467, -0.84204758, -0.01978257,  0.54494515,
        -0.65110774, -2.02695725, -0.50287533,  0.10005916],
       [-0.41601678,  1.28634273, -0.43665923,  1.35292929, -0.15339658,
         2.29114625,  1.52777467, -0.84204758,  0.32061819,  0.54494515,
        -0.65110774,  1.49580342, -1.1860961 ,  0.10005916],
       [-0.41601678,  0.61775192, -0.90653421,  1.35292929, -0.15339658,
         1.61766441,  1.52777467, -0.84204758,  0.11637774,  0.54494515,
         1.53584414,  0.90867664, -0.33207014,  0.10005916],
       [-0.41601678,  0.61775192, -0.67159672,  1.35292929, -0.15339658,
         1.16867652,  1.52777467, -0.84204758, -0.01978257,  0.54494515,
         1.53584414,  0.90

In [254]:
X_outcome_std=pd.DataFrame(X_outcome_std)
col=X_outcome.columns
X_outcome_std.columns=col
X_outcome_std.head()

,no_of_trainings,previous_year_rating,length_of_service,KPIs,awards_won,avg_training_score,department_mean,region_mean,total_points,education_lbl,gender_lbl,KPI_OTH,joining_age,Age_bins
0,-0.416017,-2.056611,-1.141472,1.352929,-0.153397,1.019014,1.527775,-0.842048,-0.065169,0.544945,-0.651108,-1.439830,-1.015291,0.100059
1,-0.416017,-2.056611,-1.141472,-0.739137,-0.153397,1.168677,1.527775,-0.842048,-0.019783,0.544945,-0.651108,-2.026957,-0.502875,0.100059
2,-0.416017,1.286343,-0.436659,1.352929,-0.153397,2.291146,1.527775,-0.842048,0.320618,0.544945,-0.651108,1.495803,-1.186096,0.100059
3,-0.416017,0.617752,-0.906534,1.352929,-0.153397,1.617664,1.527775,-0.842048,0.116378,0.544945,1.535844,0.908677,-0.332070,0.100059
4,-0.416017,0.617752,-0.671597,1.352929,-0.153397,1.168677,1.527775,-0.842048,-0.019783,0.544945,1.535844,0.908677,-0.332070,0.100059


In [255]:
y_pred_lgbm_outcome=lgbm_final.predict(X_outcome_std)
y_pred_lgbm_outcome[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [256]:
y_pred_xgb_outcome=xgb_final_1_rndm.predict(X_outcome_std)
y_pred_xgb_outcome[:10]

array([1, 1, 0, 0, 0, 1, 1, 1, 1, 0], dtype=int64)

In [259]:
df_outcome=pd.DataFrame({'employee_id':data_test['employee_id'],'is_promoted':y_pred_xgb_outcome})
df_outcome.head()

,employee_id,is_promoted
0,8724,1
1,14669,1
2,21160,0
3,4730,0
4,25879,0
